In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
from datetime import timedelta

# Functions

In [2]:
# Function to load data to Mysql
def load_MySQL(table_name,new_dataframe):

    # Replace the following with your own database connection details
    db_username = 'u681559533_AMT'
    db_password = 'MVT2024VTise'
    db_host = '193.203.166.160'
    db_port = '3306'
    db_name = 'u681559533_VT_LF_DB'

    # Create the database engine
    engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

    # Load the DataFrame into MySQL table
    new_dataframe.to_sql(name=table_name, con=engine, if_exists='replace', index=False, chunksize=500_000)

    print("DataFrame has been loaded into the database.")

    return None

# Read Data

In [3]:
root = ET.parse('UR5Schema.xml')

# Define namespace to handle the default namespace in the XML
namespaces = {'ns': 'urn:mtconnect.org:MTConnectDevices:1.7'}

# Initialize a list to store extracted data
data = []

# Iterate through each device in the XML
for device in root.findall('.//ns:Device', namespaces):
    device_info = {
        'id': device.get('id'),
        'uuid': device.get('uuid'),
        'name': device.get('name'),
        'mtconnectVersion': device.get('mtconnectVersion'),
        'manufacturer': device.find('.//ns:Description', namespaces).get('manufacturer'),
        'model': device.find('.//ns:Description', namespaces).get('model'),
        'serialNumber': device.find('.//ns:Description', namespaces).get('serialNumber'),
    }
    # Iterate through each DataItem in the device
    for data_item in device.findall('.//ns:DataItem', namespaces):
        data_item_info = {
            'dataItemId': data_item.get('id'),
            'name': data_item.get('name'),
            'category': data_item.get('category'),
            'type': data_item.get('type'),
            'subType': data_item.get('subType', 'N/A'),  # Handle missing subType with 'N/A'
            'compositionId': data_item.get('compositionId', 'N/A')  # Handle missing compositionId with 'N/A'
        }
        # Combine device info with data item info
        combined_info = {**device_info, **data_item_info}
        data.append(combined_info)

# Create DataFrame
df = pd.DataFrame(data)

In [4]:
df

,id,uuid,name,mtconnectVersion,manufacturer,model,serialNumber,dataItemId,category,type,subType,compositionId
0,UR5,UR5e,avail,1.7,UniversalRobots,UR5 CB3,2014350001,avail,EVENT,AVAILABILITY,N/A,N/A
1,UR5,UR5e,asset_changed,1.7,UniversalRobots,UR5 CB3,2014350001,asset_changed,EVENT,ASSET_CHANGED,N/A,N/A
2,UR5,UR5e,asset_removed,1.7,UniversalRobots,UR5 CB3,2014350001,asset_removed,EVENT,ASSET_REMOVED,N/A,N/A
3,UR5,UR5e,functional_mode,1.7,UniversalRobots,UR5 CB3,2014350001,functional_mode,EVENT,FUNCTIONAL_MODE,N/A,N/A
4,UR5,UR5e,compostionstate1,1.7,UniversalRobots,UR5 CB3,2014350001,gripperstate,EVENT,COMPOSITION_STATE,MOTION,GripperControl
5,UR5,UR5e,j0,1.7,UniversalRobots,UR5 CB3,2014350001,122123,SAMPLE,ANGLE,ACTUAL,N/A
6,UR5,UR5e,j1,1.7,UniversalRobots,UR5 CB3,2014350001,122223,SAMPLE,ANGLE,ACTUAL,N/A
7,UR5,UR5e,j2,1.7,UniversalRobots,UR5 CB3,2014350001,122323,SAMPLE,ANGLE,ACTUAL,N/A
8,UR5,UR5e,j3,1.7,UniversalRobots,UR5 CB3,2014350001,122121,SAMPLE,ANGLE,ACTUAL,N/A
9,UR5,UR5e,j4,1.7,UniversalRobots,UR5 CB3,2014350001,122221,SAMPLE,ANGLE,ACTUAL,N/A


# Load Data

In [5]:
print("Rows to load to MySQL:", len(df))
print("Expected time:", (len(df)/1_000_000)*1.15, "minutes")

Rows to load to MySQL: 11
Expected time: 1.265e-05 minutes


In [6]:
# Load the data to MySQL
load_MySQL('Ur5',df)

DataFrame has been loaded into the database.
